In [1]:
from ray.rllib.examples.mpt_repo.mpt.module.configuration_mpt import MPTConfig
from ray.rllib.examples.mpt_repo.mpt.module.modeling_mpt import MPTForCausalLM, MPTModel, MPTBlock
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
config = MPTConfig()

In [3]:
config.attn_config["attn_impl"] = "torch"

In [4]:
config

MPTConfig {
  "attn_config": {
    "alibi": false,
    "alibi_bias_max": 8,
    "attn_impl": "torch",
    "attn_pdrop": 0.0,
    "attn_type": "multihead_attention",
    "attn_uses_sequence_id": false,
    "clip_qkv": null,
    "prefix_lm": false,
    "qk_ln": false,
    "softmax_scale": null
  },
  "d_model": 2048,
  "emb_pdrop": 0.0,
  "embedding_fraction": 1.0,
  "expansion_ratio": 4,
  "init_config": {
    "fan_mode": "fan_in",
    "init_nonlinearity": "relu",
    "name": "kaiming_normal_"
  },
  "init_device": "cpu",
  "learned_pos_emb": true,
  "logit_scale": null,
  "max_seq_len": 2048,
  "model_type": "mpt",
  "n_heads": 16,
  "n_layers": 24,
  "no_bias": false,
  "norm_type": "low_precision_layernorm",
  "resid_pdrop": 0.0,
  "transformers_version": "4.28.1",
  "use_cache": false,
  "verbose": 0,
  "vocab_size": 50368
}

In [5]:
# model = MPTForCausalLM(config)
# model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-7b", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("mosaicml/mpt-7b-instruct", trust_remote_code=True)


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


/mnt/shared_storage/kourosh/hf_home/modules/transformers_modules/mosaicml/mpt-7b-instruct/bd1748ec173f1c43e11f1973fc6e61cb3de0f327/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [11]:
inputs = tokenizer(["Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Write a travel blog about a 3-day trip to Thailand. ### Response: "], return_tensors="pt")
inputs

{'input_ids': tensor([[30003,   310,   271,  9775,   326,  8631,   247,  4836,    15, 19566,
           247,  2380,   326, 20420, 29141,   253,  2748,    15,   209,  4118,
         41959,    27, 19566,   247,  4288,  5311,   670,   247,   495,    14,
          1201,  7408,   281, 23174,    15,   209,  4118, 19371,    27,   209]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
outputs = model.generate(
    **inputs,
    max_new_tokens=10
)
outputs

tensor([[30003,   310,   271,  9775,   326,  8631,   247,  4836,    15, 19566,
           247,  2380,   326, 20420, 29141,   253,  2748,    15,   209,  4118,
         41959,    27, 19566,   247,  4288,  5311,   670,   247,   495,    14,
          1201,  7408,   281, 23174,    15,   209,  4118, 19371,    27,   209,
           187, 15490,   337,    27, 48771,   187,  2906, 30487,   275, 48771]])

In [13]:
output_text = tokenizer.decode(outputs[0])
output_text

'Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Write a travel blog about a 3-day trip to Thailand. ### Response: \nDay 1: Bangkok\nArrived in Bangkok'